In [1]:
import re
import requests
import urllib.request as img_request
import time
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from os import makedirs
from os import listdir
from os import getcwd
import os

from sklearn.model_selection import train_test_split

# Init

Defining important directories as variables

In [2]:
cwd = getcwd()
data_dir = "data/"
img_dir = "data/images/"

Defining regex tags

In [ ]:
n_pages_tag = re.compile(r'<a href=.*page=(\d*).*/li>') # For getting the amount of pages on the domain

name_tag = re.compile(r'<a href=\"https://sailboatdata\.com/sailboat/.*\">(.*)</a>') # For making a list of all the boats on sailboatdata

specs_tag = re.compile(r'<div class=\" col-\w\w-\d*  col-\w\w-6 sailboatdata-label \">\s*(.*):\s</div>\s<.*\s*(.*)') # For scraping data from specific boat url

image_tag = re.compile(r'(?:photo|drawing)\".*src=\"(http.*)\"/>')

photo_draw_tag = re.compile(r'sailboat/(\w*)')

In [ ]:
if not (os.path.exists(data_dir)):
    print("Making dir" , data_dir)
    makedirs(data_dir)

if not(os.path.exists(img_dir)):
    print("Making dir" , img_dir)
    makedirs(img_dir)

# Scraping

In [ ]:
specs = []
categories = []
boat_data = pd.DataFrame()
counter = 0

In [ ]:
general_url = 'https://sailboatdata.com/sailboat?page={}&paginate=25'

print("Connecting to" , general_url.format(1))
init_request = requests.get(general_url.format(1))
print(init_request.status_code)
pages = re.findall(n_pages_tag , init_request.text)
n_pages = pages[-1]

boat_types = []
for i in tqdm(range (1,int(n_pages)+1)):
  # print("Connecting to" , general_url.format(i))
  r = requests.get(general_url.format(i))
  # print(r.status_code)
  boat_types += re.findall(name_tag,r.text)
  time.sleep(0.5) #To not throw too many requests at the website

print(len(boat_types))

In [ ]:
for boat in boat_types:
    # GETTNG REQUEST
    print("Scraping",boat,"...")
    r = requests.get("https://sailboatdata.com/sailboat/{}?units=metric".format(boat.replace(" ","-").replace("(","").replace(")","").replace(".","")))
    raw_text = r.text

    # SCRAPING IMAGES
    images = re.findall(image_tag,raw_text)
    print("         -------scraping")
    for image in images:
        img_path = str(counter).zfill(5)+".jpg"
        if not os.path.isfile(img_path):
            counter += 1
            print(image)
            try:
                img_request.urlretrieve(image,img_dir+img_path)
            except:
                print("Strange url" , image.replace(" ","%20"))
                try:
                    img_request.urlretrieve(image,img_dir+img_path)
                except:
                    has_image = False

            # SCRAPING SPECS
            categories_specs = re.findall(specs_tag,raw_text)
            specs = [el[1] for el in categories_specs]
            categories = [el[0] for el in categories_specs]
            new_row = {categories[i]: specs[i] for i in range(len(categories))}
            new_row['name'] = boat
            new_row['img_path'] = img_path
            boat_data = pd.concat([boat_data, pd.DataFrame([new_row])], ignore_index=True)



    print("Done\n")
    if counter%100 == 1:
        print("saving")
        boat_data.to_csv(data_dir+"boat_data.csv" , index=False)
boat_data.to_csv(data_dir+"boat_data.csv" , index = False)

In [ ]:
pd.set_option('display.max_columns', None)
data_raw = pd.read_csv(data_dir+"boat_data.csv")
data_raw

# Cleaning

In [ ]:
units_cleaner_tag = re.compile(r'([\d.]+)(\s*m|\s*kg)(<.*>)*')

def clean_row(row):
  cleaned_row = []
  for i in range(len(row)):
    if i != 32:
      try:
        cleaned_row.append(float(re.sub(units_cleaner_tag , r'\g<1>' , row[i].replace(',','') , )))
      except:
        cleaned_row.append(row[i])
    else:
      cleaned_row.append(row[i])
  return cleaned_row

In [ ]:
cleaned_data = []
for i , row in data_raw.iterrows():
  cleaned_data.append(clean_row(row))

In [ ]:
data_clean = pd.DataFrame(cleaned_data , columns = data_raw.columns)
data_clean.to_csv(data_dir+"boat_data_clean.csv" , index = False)

In [10]:
data_clean = pd.read_csv(data_dir+"boat_data_clean.csv")

In [11]:
for column in data_clean.columns:
    if (data_clean[column].dtype == "object" or data_clean[column].dtype == "category") and not column == "Download Boat Record" and not column == "url" and not column == "img_path":
        data_clean[column] = data_clean[column].str.strip().str.rstrip('.').str.lower()
        # data_clean[column] = data_clean[column].astype('category')
        # pd.DataFrame(data_clean[column].cat.categories).to_csv(data_dir+"labels/"+column.replace(" ","_")+".txt" , index = False , header = False)
        # data_clean[column] = data_clean[column].cat.codes

# Splitting

In [12]:
data_train , data_test = train_test_split(data_clean , test_size = 0.2 , random_state = 42)


In [ ]:
train_dir = "data/train/"
test_dir = "data/test/"

if not(os.path.exists(train_dir)):
    print("Making dir" , train_dir)
    makedirs(train_dir)

if not(os.path.exists(test_dir)):
    print("Making dir" , test_dir)
    makedirs(test_dir)


for image in data_train['img_path']:
    try:
        img = Image.open(img_dir+image)
        img = img.convert('RGB')
        img.save(train_dir+image)
    except:
        print("Image not found" , image)
        data_train = data_train[data_train['img_path'] != image]

for image in data_test['img_path']:
    try:
        img = Image.open(img_dir+image)
        img = img.convert('RGB')
        img.save(test_dir+image)
    except:
        print("Image not found" , image)
        data_test = data_test[data_test['img_path'] != image]


In [ ]:
data_train.to_csv(data_dir+"boat_data_train.csv" , index = False)
data_test.to_csv(data_dir+"boat_data_test.csv" , index = False)

# Plotting

In [6]:
# data_train = pd.read_csv(data_dir+"boat_data_train.csv")
# data_test = pd.read_csv(data_dir+"boat_data_test.csv")
# data_clean = pd.read_csv(data_dir+"boat_data_clean.csv")

In [14]:
plot_dir = "plots/"
if not(os.path.exists(plot_dir)):
    print("Making dir" , plot_dir)
    os.makedirs(plot_dir)

In [15]:
train_test = ["train" , "test"]
size_factor = [1,0.2]
for i , data in enumerate([data_train , data_test]):
    for column in data_clean.columns:
        if not column == "Download Boat Record" and not column == "name" and not column == "url" and not column == "img_path":
            fig = plt.figure(figsize=(10*2,5*2))
            if data[column].dtype == "object":
                to_remove = 50*size_factor[i]
                try:
                    dat = data[column].value_counts()
                    # dat = dat[dat > to_remove] # Removing entries with less than to_remove entries
                    dat = dat/len(data[column]) # Normalising
                    print("Plotting" , column)
                    dat.to_csv(plot_dir+column.replace('/',"_")+"_"+train_test[i]+".csv")
                    dat.plot(kind='barh')
                    plt.annotate(text = "Total entries were "+str(len(data[column].value_counts())) , xy = (0.8,0.95) , xycoords = "axes fraction")
                    plt.annotate(text = "Removed entries where n<"+str(to_remove) , xy = (0.8,0.9) , xycoords = "axes fraction")
                except:
                    data[column].value_counts().plot(kind='barh')
                fig.name = column
                fig.suptitle(column+str(" ")+train_test[i] , fontsize=16)
                path = plot_dir+column.replace('/',"_")+"_"+train_test[i]+".png"
                plt.savefig(path)
                fig.clear()


            # else:
                # data[column].plot(kind='hist' , logy=True)
            # fig.name = column
            # fig.suptitle(column+str(" ")+train_test[i] , fontsize=16)
            # path = plot_dir+column.replace('/',"_")+"_"+train_test[i]+".png"
            # plt.savefig(path)
            # fig.clear()


Plotting Hull Type
Plotting Rigging Type
Plotting Construction
Plotting Ballast Type
Plotting Designer


C:\Users\chris\AppData\Local\Temp\ipykernel_5516\2040015531.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10*2,5*2))


Plotting Builder
Plotting Model
Plotting Make
Plotting Type
Plotting HP
Plotting Fuel
Plotting Water
Plotting Website
Plotting Hull Type
Plotting Rigging Type
Plotting Construction
Plotting Ballast Type
Plotting Designer
Plotting Builder
Plotting Model
Plotting Make
Plotting Type
Plotting HP
Plotting Fuel
Plotting Water
Plotting Website


<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>

...

<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>

<Figure size 2000x1000 with 0 Axes>

# Label embedding

In [ ]:
data_clean = pd.DataFrame(cleaned_data , columns = data_raw.columns)

for column in data_clean.columns:
    if data_clean[column].dtype == "object" and not column == "Download Boat Record" and not column == "url" and not column == "img_path":
        data_clean[column] = data_clean[column].astype('category')
        pd.DataFrame(data_clean[column].cat.categories).to_csv(data_dir+column+".txt" , index = False , header = False)
        data_clean[column] = data_clean[column].cat.codes

data_clean.to_csv(data_dir+"boat_data_clean.csv" , index = False)